In [91]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import os
import sys
from ocp_table_tpot.globals import Globals as gd
from tpot import TPOTRegressor
sys.path.insert(0,'..')
from src.models.model import HistoricalMedian,XGBoost,LinearModel,RF,KNN,SVM,mase,TimeSeriesSplitImproved
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from skgarden.quantile import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.preprocessing import FunctionTransformer
from copy import copy
from tpot.builtins import StackingEstimator
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline, make_union
from catboost import CatBoostRegressor,Pool,cv


df_tsfresh = pd.read_pickle(f'../data/processed/train_test_tsfresh.pkl').reset_index(level = 0)
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')

In [104]:
year = 2019
tgt = 'final.output.recovery'
X = data_dict[year]['X_train_ts']
y = data_dict[year]['y_train']
X_test=  data_dict[year]['X_test']
print(f'1) Test shape: {X_test.shape}, train: {X.shape}')
    
inds_y = y[(y[tgt] > 5) & (y[tgt] < 100)].index
inds_common = inds_y

X = X.loc[inds_common,]
y = y.loc[inds_common, tgt]

X = X.sample(frac=0.3,random_state=123).sort_index().dropna()
y= y[X.index]



Nmonths_total = 8
Nspl = int(Nmonths_total * 30 / 15)
Nmonths_test = 4
Nmonths_min_train = 2.5
train_splits = Nspl // Nmonths_total*Nmonths_min_train
test_splits=int(Nmonths_test / Nmonths_total*Nspl)
cv = TimeSeriesSplitImproved(n_splits=Nspl)

1) Test shape: (5856, 53), train: (16859, 153)


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\pandas\core\sorting.py:257: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  items = np.asanyarray(items)


# Base models

In [111]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.005, random_state=1))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.005, l1_ratio=.9, random_state=3))

CatBoost = CatBoostRegressor(loss_function='MAE',random_seed =123,learning_rate=0.1,max_depth=8,task_type='GPU',od_type = 'Iter',od_wait= 15,iterations = 2000)

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=12, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.15, max_depth=12, 
                             n_estimators=500,
                             reg_alpha=0.4, reg_lambda=0.8,
                             subsample=0.5, silent=1,
                             random_state =7, nthread = -1)

model_lgb = lgb.LGBMRegressor(objective='mae',num_leaves=5,
                              learning_rate=0.15, n_estimators=500,
                              bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9  )
param_grids = {'n_estimators': 1000,
                   'max_features': 0.8, # tuned
                   'max_depth': 14, # tuned
                   }
model_qrf = RandomForestQuantileRegressor(**param_grids,
               criterion = 'mae',
               n_jobs = -1,
                random_state =123)

In [106]:
n_folds = 5

def rmsle_cv(model,cv = None):
    if cv is None:
        cv = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    score  = make_scorer(mase,greater_is_better=False)
    #rmse= np.sqrt(-cross_val_score(model, X.values, y.values.reshape(-1,1), scoring=score, cv = cv))
    rmse= cross_val_score(model, X.values, y.values.reshape(-1,), scoring=score, cv = cv)
    
    return(rmse)

def rmsle_cv_boxcox_tgt(model,cv = None,lam = 4.321):
    from scipy.stats import boxcox
    from scipy.special import inv_boxcox
    if cv is None:
        cv = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    scores = []
    for fold_n, (train_index, valid_index) in enumerate(cv.split(X,fixed_length=False, train_splits=train_splits, test_splits=test_splits)):
    # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        y_train = boxcox(y_train,lam)
        model.fit(X_train,y_train)
        preds  = inv_boxcox(model.predict(X_valid),4.321)
        score_val = mase(preds,y_valid)
        # print(f'Fold {fold_n}. Score: {score_val:.4f}.')
        print('')
        scores.append(score_val)
    
    #print(f'CV mean score: {np.mean(scores):.4f}, std: {np.std(scores):.4f}.')
    
    return(np.array(scores))


class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [107]:
score = rmsle_cv(lasso,cv = cv)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3


Lasso score: -3.1228 (0.3227)



C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3


Lasso score: -2.9564 (0.4491)



In [108]:
score = rmsle_cv(ENet,cv=cv)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(ENet,cv=None)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

ElasticNet score: -3.1416 (0.3207)



C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

ElasticNet score: -2.9653 (0.4515)



In [98]:
score = rmsle_cv(GBoost,cv=cv)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

KeyboardInterrupt: 

In [109]:
score = rmsle_cv(model_xgb,cv=cv)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: -2.5367 (0.2999)

Xgboost score: -2.1219 (0.1754)



In [110]:
score = rmsle_cv(model_lgb,cv=cv)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: -3.5367 (0.2241)

LGBM score: -3.0009 (0.2294)



In [104]:
score = rmsle_cv(model_qrf,cv=cv)
print("QRFscore: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
score = rmsle_cv(model_qrf)
print("QRF score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: -3.4970 (0.6479)



KeyboardInterrupt: 

In [ ]:
pool = Pool(x_train, y_train)
params = {'iterations': 100, 
          'depth': 2, 
          'loss_function': 'MultiClass', 
          'classes_count': 3, 
          'verbose': False}
scores = cv(pool, params)



0:	learn: 64.1853531	total: 43.2ms	remaining: 1m 26s
1:	learn: 64.1366830	total: 91.9ms	remaining: 1m 31s
2:	learn: 64.0876312	total: 116ms	remaining: 1m 17s
3:	learn: 64.0388359	total: 165ms	remaining: 1m 22s
4:	learn: 63.9894024	total: 181ms	remaining: 1m 12s
5:	learn: 63.9404342	total: 212ms	remaining: 1m 10s
6:	learn: 63.8908874	total: 226ms	remaining: 1m 4s
7:	learn: 63.8421279	total: 272ms	remaining: 1m 7s
8:	learn: 63.7932073	total: 317ms	remaining: 1m 10s
9:	learn: 63.7436605	total: 331ms	remaining: 1m 5s
10:	learn: 63.6946446	total: 359ms	remaining: 1m 4s
11:	learn: 63.6460997	total: 402ms	remaining: 1m 6s
12:	learn: 63.5972507	total: 431ms	remaining: 1m 5s
13:	learn: 63.5484971	total: 474ms	remaining: 1m 7s
14:	learn: 63.4993201	total: 496ms	remaining: 1m 5s
15:	learn: 63.4504175	total: 538ms	remaining: 1m 6s
16:	learn: 63.4013359	total: 559ms	remaining: 1m 5s
17:	learn: 63.3527195	total: 600ms	remaining: 1m 6s
18:	learn: 63.3034709	total: 618ms	remaining: 1m 4s
19:	learn: 63

In [122]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, lasso,model_xgb),
                                                 meta_model = lasso)
score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

Stacking Averaged models score: -3.0926 (0.5857)


# Pull the TPOT models as basic models, attach QRF and do a stacking prediction

In [90]:
pipe_a = exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        MinMaxScaler()
    ),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=5, max_features=0.15000000000000002, min_samples_leaf=0.055, min_samples_split=0.505, n_estimators=100)),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=False, max_depth=9, max_features=0.15000000000000002, min_samples_leaf=0.255, min_samples_split=0.005, n_estimators=100)),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=5, max_features=0.35000000000000003, min_samples_leaf=0.20500000000000002, min_samples_split=0.35500000000000004, n_estimators=250)),
    LGBMRegressor(colsample_bytree=0.75, learning_rate=0.01, max_bin=127, max_depth=4, min_child_weight=15, n_estimators=300, num_leaves=90, objective="fair", reg_alpha=0.05, subsample=0.75, subsample_freq=0, verbosity=-1,n_thread=-1)
)

pipe_b = make_pipeline(
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=4, max_features=0.5500000000000002, min_samples_leaf=0.055, min_samples_split=0.455, n_estimators=300)),
    MinMaxScaler(),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=5, max_features=0.5500000000000002, min_samples_leaf=0.255, min_samples_split=0.20500000000000002, n_estimators=500)),
    LGBMRegressor(colsample_bytree=1.0, learning_rate=0.2, max_bin=63, max_depth=4, min_child_weight=0.001, n_estimators=250, num_leaves=90, objective="huber", reg_alpha=0.007, subsample=0.8, subsample_freq=0, verbosity=-1,n_thread=-1)
)

pipe_c = exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        make_union(
            FunctionTransformer(copy),
            FunctionTransformer(copy)
        )
    ),
    StackingEstimator(estimator=LGBMRegressor(colsample_bytree=1.0, learning_rate=0.005, max_bin=127, max_depth=5, min_child_weight=1, n_estimators=250, num_leaves=100, objective="mape", reg_alpha=0.01, subsample=0.7, subsample_freq=10, verbosity=-1)),
    LGBMRegressor(colsample_bytree=1.0, learning_rate=0.001, max_bin=127, max_depth=4, min_child_weight=10, n_estimators=300, num_leaves=70, objective="mape", reg_alpha=0.05, subsample=0.9, subsample_freq=30, verbosity=-1,n_thread=-1)
)


stacked_averaged_models = StackingAveragedModels(base_models = (ENet, pipe_a, pipe_b,pipe_c,lasso),
                                                 meta_model = ENet)

score = rmsle_cv(stacked_averaged_models,cv=cv)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


In [ ]:
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))
print("\nBoxCox Stacked score: {:.4f} ({:.4f})\n".format(score_bc.mean(), score_bc.std()))

# Final predictions:



In [75]:
from scipy.stats import boxcox
from scipy.special import inv_boxcox
preds_all = []

for year in [2019]:
    print(year)
    data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')
    
    tgt = "rougher.output.recovery"
    X = data_dict[year]['X_train_ts'].copy()
    X_test = data_dict[year]['X_test_ts'].copy()
    print(f'1) Test shape: {X_test.shape}, train: {X.shape}')
    y = data_dict[year]['y_train'][tgt].dropna()
    y = y[(y>5) & (y <100)]
    inds = X.index.intersection(y.index)
    X = X.loc[inds]
    y = y.loc[inds]
        
    stacked_averaged_models.fit(X.values, y)
    ypred_r = stacked_averaged_models.predict(X_test.values)
    
    preds_r = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_r}).set_index('date')
    
    tgt = "final.output.recovery"
    print(f'2) Test shape: {X_test.shape}, train: {X.shape}')
    
    X = data_dict[year]['X_train_ts'].copy()
    X_test = data_dict[year]['X_test_ts'].copy()
    y = data_dict[year]['y_train'][tgt].dropna()
    y = y[(y>5) & (y <100)]
    inds = X.index.intersection(y.index)
    X = X.loc[inds]
    y = y.loc[inds]
    print(f'3) Test shape: {X_test.shape}, train: {X.shape}')
    
    y = boxcox(y,lam)
    stacked_averaged_models.fit(X.values, y)
    ypred_f = stacked_averaged_models.predict(X_test.values)
    ypred_f = inv_boxcox(ypred_f,4.321)
    preds_f = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_f}).set_index('date')

    preds_all.append(preds_r.join(preds_f))

2016
1) Test shape: (2928, 153), train: (5520, 153)


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

2) Test shape: (2928, 153), train: (4691, 153)
3) Test shape: (2928, 153), train: (4901, 153)


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

2017
1) Test shape: (2928, 153), train: (5832, 153)


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

2) Test shape: (2928, 153), train: (4951, 153)
3) Test shape: (2928, 153), train: (5218, 153)


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

In [76]:
stacked_preds_sub = pd.concat(preds_all)
stacked_preds_sub = stacked_preds_sub.reset_index()
stacked_preds_sub['date'] = stacked_preds_sub['date'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
stacked_preds_sub.set_index('date',inplace=True)
stacked_preds_sub.to_csv('../results/stacked_sub_3moredata.csv')

(5856, 2)

In [124]:
ypred_r

array([89.82622333, 89.75616916, 89.5315029 , ..., 81.3202456 ,
       80.8921535 , 81.34507872])